In [ ]:
import pymc as pm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arviz as az
import os

# === Load log returns data ===
df = pd.read_csv('../data/processed/brent_log_returns.csv', parse_dates=['Date'])
df.dropna(inplace=True)

returns = df['Log_Return'].values
n = len(returns)

# === Bayesian Change Point Model ===
with pm.Model() as model:
    # Use a narrower range for tau to reduce computation
    tau = pm.DiscreteUniform('tau', lower=int(n * 0.3), upper=int(n * 0.7))

    # Priors for mean before and after change
    mu1 = pm.Normal('mu1', mu=0, sigma=0.5)
    mu2 = pm.Normal('mu2', mu=0, sigma=0.5)

    # Shared standard deviation with tighter prior
    sigma = pm.HalfNormal('sigma', sigma=0.5)

    # Define piecewise mean based on tau
    idx = np.arange(n)
    mu = pm.Deterministic("mu", pm.math.switch(idx < tau, mu1, mu2))

    # Likelihood
    obs = pm.Normal('obs', mu=mu, sigma=sigma, observed=returns)

    # Use Slice sampler for tau (more efficient for discrete variables)
    step1 = pm.Slice(vars=[tau])
    step2 = pm.NUTS(vars=[mu1, mu2, sigma], target_accept=0.9)

    # Reduced sampling for faster execution
    trace = pm.sample(
        draws=100,
        tune=100,
        chains=1,
        cores=1,
        step=[step1, step2],
        return_inferencedata=True
    )

# === Save diagnostics summary ===
os.makedirs('../outputs/logs', exist_ok=True)
summary = az.summary(trace, var_names=["mu1", "mu2", "tau", "sigma"])
summary.to_csv('../outputs/logs/trace_summary.csv')
print(summary)

# === Plot traces ===
os.makedirs('../outputs/figures', exist_ok=True)
az.plot_trace(trace, var_names=["mu1", "mu2", "tau", "sigma"])
plt.savefig('../outputs/figures/change_point_traceplot.png')
plt.close()

# === Identify most probable change point ===
most_probable_tau = int(trace.posterior['tau'].values[0].mean())
change_date = df.iloc[most_probable_tau]['Date']
print(f"\n📍 Most probable change point date: {change_date.date()}")

# === Quantify the impact of the change ===
mu1_samples = trace.posterior['mu1'].values.flatten()
mu2_samples = trace.posterior['mu2'].values.flatten()
delta = mu2_samples - mu1_samples

print(f"\nEstimated Mean Before Change: {np.mean(mu1_samples):.5f}")
print(f"Estimated Mean After Change:  {np.mean(mu2_samples):.5f}")
print(f"Change in Mean:               {np.mean(delta):.5f}")
print(f"Probability After > Before:   {np.mean(mu2_samples > mu1_samples):.2%}")



# === Posterior distributions plot ===
plt.figure(figsize=(8, 5))
plt.hist(mu1_samples, bins=30, alpha=0.5, label='mu1 (Before)', color='skyblue')
plt.hist(mu2_samples, bins=30, alpha=0.5, label='mu2 (After)', color='orange')
plt.axvline(np.mean(mu1_samples), color='blue', linestyle='--')
plt.axvline(np.mean(mu2_samples), color='red', linestyle='--')
plt.legend()
plt.title('Posterior Distributions of Mean Log Returns')
plt.xlabel('Mean Log Return')
plt.ylabel('Frequency')
plt.savefig('../outputs/figures/posterior_mu_comparison.png')
plt.close()

Sequential sampling (1 chains in 1 job)
CompoundStep
>Slice: [tau]
>NUTS: [mu1, mu2, sigma]


c:\Users\123\Desktop\Kifiya AI & ML\week-10\brent-oil-analysis\venv\Lib\site-packages\rich\live.py:256: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

c:\Users\123\Desktop\Kifiya AI & 
ML\week-10\brent-oil-analysis\venv\Lib\site-packages\pytensor\scalar\basic.py:2094: RuntimeWarning: overflow 
encountered in divide
  return x / y

c:\Users\123\Desktop\Kifiya AI & 
ML\week-10\brent-oil-analysis\venv\Lib\site-packages\pytensor\scalar\basic.py:3297: RuntimeWarning: overflow 
encountered in scalar multiply
  return x * x

c:\Users\123\Desktop\Kifiya AI & 
ML\week-10\brent-oil-analysis\venv\Lib\site-packages\pytensor\tensor\elemwise.py:1411: RuntimeWarning: invalid 
value encountered in reduce
  out = self.ufunc.reduce(input, axis=axis, dtype=acc_dtype)

c:\Users\123\Desktop\Kifiya AI & 
ML\week-10\brent-oil-analysis\venv\Lib\site-packages\pytensor\scalar\basic.py:2094: RuntimeWarning: divide by zero 
encountered in divide
  return x / y

Sampling 1 chain for 100 tune and 100 draw iterations (100 + 100 draws total) took 2400 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks
arviz - WARNING - Shape validation failed: input_shape: (1, 100), minimum_shape: (chains=2, draws=4)


           mean       sd    hdi_3%   hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
mu1       0.000     0.00    -0.000     0.001      0.000    0.000     120.0   
mu2       0.000     0.00    -0.000     0.001      0.000    0.000      62.0   
tau    4622.550  1005.56  3053.000  6256.000    130.414   73.849      62.0   
sigma     0.026     0.00     0.025     0.026      0.000    0.000      27.0   

       ess_tail  r_hat  
mu1        77.0    NaN  
mu2        77.0    NaN  
tau        59.0    NaN  
sigma      21.0    NaN  

📍 Most probable change point date: 2005-07-19

Estimated Mean Before Change: 0.00022
Estimated Mean After Change:  0.00013
Change in Mean:               -0.00009
Probability After > Before:   45.00%
